In [1]:
import os
import json
import jsonlines
from collections import defaultdict
import time

In [2]:
all_start = time.time()

In [3]:
# CMeIE
# A. c→(h,t)
out_name = 'c_to_ht_withtype'
inp_prompt = '''当前你是一个资深的信息提取的专家。
你的任务是从给定文本中，抽取所有可能的主客体实体对。先抽取可能存在的主体跨度，再基于抽取的主体跨度和给定文本继续抽取对应的客体跨度。并分别基于主/客体从给定实体列表中提取主/客体的实体类型。
给定的实体类型列表：{entity_list}。
任务的输出形式是（主体||主体类型||客体||客体类型）。
给定文本：“{text}”
'''
oup_prompt = '{answer_text}'

In [4]:
data_file_list = [
    'CMeIE-V2_train.jsonl',
    'CMeIE-V2_dev.jsonl',
    'CMeIE-V2_test.jsonl',
]
out_file_list = [
    'train.json',
    'dev.json',
    'test.json'
]

In [5]:
read_dir = './ori/CMeIE-V2/'
out_dir = './pipeline/CMeIE-V2'
out_dir = os.path.join(out_dir,out_name)
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
with open(os.path.join(read_dir,'53_schemas.json'),'r') as f:
    trip_types_list = json.load(f)

In [6]:
trip_types = sorted(set([data['predicate'] for data in trip_types_list]))
ent_type_list = []
for data in trip_types_list:
    ent_type_list.append(data['subject_type'])
    ent_type_list.append(data['object_type'])
ent_types = sorted(set(ent_type_list),reverse=True)

In [7]:
start = time.time()
print('out_name:{}'.format(out_name))
for data_file,out_file in zip(data_file_list,out_file_list):
    read_path = os.path.join(read_dir,data_file)
    out_path = os.path.join(out_dir,out_file)
    with jsonlines.open(read_path,'r') as f:
        datas = [data for data in f]
    with jsonlines.open(out_path,'w') as fw:
        for data in datas:
            inp = inp_prompt.format(relation_list='，'.join(trip_types),entity_list = '，'.join(ent_types),text=data['text'])
            spo_list = [(item['subject'],item['subject_type'],item['object']['@value'], item['object_type']['@value']) for item in data['spo_list']]
            processed_spo_list = []
            for spo_item in spo_list:
                if spo_item not in processed_spo_list:
                    processed_spo_list.append(spo_item)
            oup = '\n'.join(['（{}||{}||{}||{}）'.format(item[0],item[1],item[2], item[3]) for item in processed_spo_list])
            oup = '```\n' + oup.strip() + '\n```'
            out_data = {
                'instruction':inp,
                'input':'',
                'output':oup,
                'text':data['text'],
                'spo_list':data['spo_list']
            }
            fw.write(out_data)
end = time.time()
print('cost:{}秒'.format(round(end-start, 2)))

out_name:c_to_ht_withtype
cost:1.29秒


In [8]:
# CMeIE
# A. c→(r)
out_name = 'c_to_r_withtype'
inp_prompt = '''当前你是一个资深的关系分类专家。
你的任务是基于给定文本，从给定关系列表中确定可能的关系。给定的关系列表：{relation_list}。
任务的输出形式是（关系类型）。
给定文本：“{text}”
'''
oup_prompt = '{answer_text}'

In [9]:
data_file_list = [
    'CMeIE-V2_train.jsonl',
    'CMeIE-V2_dev.jsonl',
    'CMeIE-V2_test.jsonl',
]
out_file_list = [
    'train.json',
    'dev.json',
    'test.json'
]

In [10]:
read_dir = './ori/CMeIE-V2/'
out_dir = './pipeline/CMeIE-V2'
out_dir = os.path.join(out_dir,out_name)
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
with open(os.path.join(read_dir,'53_schemas.json'),'r') as f:
    trip_types_list = json.load(f)

In [11]:
trip_types = sorted(set([data['predicate'] for data in trip_types_list]))
ent_type_list = []
for data in trip_types_list:
    ent_type_list.append(data['subject_type'])
    ent_type_list.append(data['object_type'])
ent_types = sorted(set(ent_type_list),reverse=True)

In [12]:
start = time.time()
print('out_name:{}'.format(out_name))
for data_file,out_file in zip(data_file_list,out_file_list):
    read_path = os.path.join(read_dir,data_file)
    out_path = os.path.join(out_dir,out_file)
    with jsonlines.open(read_path,'r') as f:
        datas = [data for data in f]
    with jsonlines.open(out_path,'w') as fw:
        for data in datas:
            inp = inp_prompt.format(relation_list='，'.join(trip_types),entity_list = '，'.join(ent_types),text=data['text'])
            spo_list = []
            for item in data['spo_list']:
                if item['predicate'] not in spo_list:
                    spo_list.append(item['predicate'])
            processed_spo_list = []
            for spo_item in spo_list:
                if spo_item not in processed_spo_list:
                    processed_spo_list.append(spo_item)
            oup = '\n'.join(['（{}）'.format(item) for item in processed_spo_list])
            oup = '```\n' + oup.strip() + '\n```'
            out_data = {
                'instruction':inp,
                'input':'',
                'output':oup,
                'text':data['text'],
                'spo_list':data['spo_list']
            }
            fw.write(out_data)
end = time.time()
print('cost:{}秒'.format(round(end-start, 2)))

out_name:c_to_r_withtype
cost:1.9秒


In [13]:
# CMeIE
# B.  r[s1] c→(h,t)
out_name = 'rc_to_ht_withtype'
inp_prompt = '''当前你是一个资深的信息提取的专家。
你的任务是从给定文本和关系类型，抽取所有可能的主客体实体对。先抽取可能存在的主体跨度，再基于抽取的主体跨度和给定文本继续抽取对应的客体跨度。并分别基于主/客体从给定实体列表中提取主/客体的实体类型。
给定的实体类型列表：{entity_list}。
任务的输出形式是（主体||主体类型||客体||客体类型）。
给定文本：“{text}”
给定关系类型：“{relation}”
'''
oup_prompt = '{answer_text}'

In [14]:
data_file_list = [
    'CMeIE-V2_train.jsonl',
    'CMeIE-V2_dev.jsonl',
    'CMeIE-V2_test.jsonl',
]
out_file_list = [
    'train.json',
    'dev.json',
    'test.json'
]

In [15]:
read_dir = './ori/CMeIE-V2/'
out_dir = './pipeline/CMeIE-V2'
out_dir = os.path.join(out_dir,out_name)
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
with open(os.path.join(read_dir,'53_schemas.json'),'r') as f:
    trip_types_list = json.load(f)

In [16]:
trip_types = sorted(set([data['predicate'] for data in trip_types_list]))

In [17]:
start = time.time()
print('out_name:{}'.format(out_name))
for data_file,out_file in zip(data_file_list,out_file_list):
    read_path = os.path.join(read_dir,data_file)
    out_path = os.path.join(out_dir,out_file)
    with jsonlines.open(read_path,'r') as f:
        datas = [data for data in f]
    with jsonlines.open(out_path,'w') as fw:
        for data in datas:
            relation_to_ht = defaultdict(list)
            for spo_item in data['spo_list']:
                sub = spo_item['subject']
                sub_type = spo_item['subject_type']
                predicate = spo_item['predicate']
                obj = spo_item['object']['@value']
                obj_type = spo_item['object_type']['@value']
                relation_to_ht[predicate].append((sub, sub_type, obj, obj_type))
            for predicate in relation_to_ht.keys():
                spo_list = relation_to_ht[predicate]
                inp = inp_prompt.format(relation_list='，'.join(trip_types),entity_list = '，'.join(ent_types),text=data['text'], relation = predicate)
                oup = '\n'.join(['（{}||{}||{}||{}）'.format(item[0],item[1],item[2], item[3]) for item in spo_list])
                oup = '```\n' + oup.strip() + '\n```'
                out_data = {
                    'instruction':inp,
                    'input':'',
                    'output':oup,
                    'text':data['text'],
                    'spo_list':data['spo_list']
                }
                fw.write(out_data)
end = time.time()
print('cost:{}秒'.format(round(end-start, 2)))

out_name:rc_to_ht_withtype
cost:2.73秒


In [18]:
# CMeIE
# C. h[s1]t [s2]c→r
out_name = 'htc_to_r_withtype'
inp_prompt = '''当前你是一个资深的关系分类专家。
你的任务是基于给定文本和主客体实体对以及他们的类型，从给定关系列表中确定可能的关系。
给定的关系列表：{relation_list}。
任务输入的实体对格式是（主体||主体类型||客体||客体类型）。
任务的输出形式是（关系类型）。
给定文本：“{text}”
给定主客体实体对：“{head_tail}”
'''
oup_prompt = '{answer_text}'

In [19]:
data_file_list = [
    'CMeIE-V2_train.jsonl',
    'CMeIE-V2_dev.jsonl',
    'CMeIE-V2_test.jsonl',
]
out_file_list = [
    'train.json',
    'dev.json',
    'test.json'
]

In [20]:
read_dir = './ori/CMeIE-V2/'
out_dir = './pipeline/CMeIE-V2'
out_dir = os.path.join(out_dir,out_name)
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
with open(os.path.join(read_dir,'53_schemas.json'),'r') as f:
    trip_types_list = json.load(f)

In [21]:
trip_types = sorted(set([data['predicate'] for data in trip_types_list]))

In [22]:
start = time.time()
print('out_name:{}'.format(out_name))
for data_file,out_file in zip(data_file_list,out_file_list):
    read_path = os.path.join(read_dir,data_file)
    out_path = os.path.join(out_dir,out_file)
    with jsonlines.open(read_path,'r') as f:
        datas = [data for data in f]
    with jsonlines.open(out_path,'w') as fw:
        for data in datas:
            ht_to_relation = defaultdict(list)
            for spo_item in data['spo_list']:
                sub = spo_item['subject']
                sub_type = spo_item['subject_type']
                predicate = spo_item['predicate']
                obj = spo_item['object']['@value']
                obj_type = spo_item['object_type']['@value']
                ht_item = (sub, sub_type, obj, obj_type)
                ht_to_relation[ht_item].append(predicate)
            for ht_item in ht_to_relation.keys():
                predicate = ht_to_relation[ht_item]
                ht_item_str = '（{}||{}||{}||{}）'.format(ht_item[0], ht_item[1], ht_item[2], ht_item[3])
                inp = inp_prompt.format(relation_list='，'.join(trip_types),text=data['text'], head_tail = ht_item_str)
                oup = '\n'.join(['（{}）'.format(item) for item in predicate])
                oup = '```\n' + oup.strip() + '\n```'
                out_data = {
                    'instruction':inp,
                    'input':'',
                    'output':oup,
                    'text':data['text'],
                    'spo_list':data['spo_list']
                }
                fw.write(out_data)
end = time.time()
print('cost:{}秒'.format(round(end-start, 2)))

out_name:htc_to_r_withtype
cost:6.17秒


In [23]:
# CMeIE
# A0. c→(h,r,t)
out_name = 'c_to_hrt_withtype'
inp_prompt = '''当前你是一个资深的信息提取的专家。
你的任务是从给定文本中，抽取关系三元组。先从给定文本中抽取主客体实体对，并基于主客体实体对从给定实体列表中提取主客体的实体类型。给定的实体类型列表：{entity_list}。
然后基于主客实体对及其对应的实体类型从给定关系列表中提取可能的关系。给定的关系列表：{relation_list}。
任务的输出形式是（主体||主体类型||关系类型||客体||客体类型）。
给定文本：“{text}”
'''
oup_prompt = '{answer_text}'

In [24]:
data_file_list = [
    'CMeIE-V2_train.jsonl',
    'CMeIE-V2_dev.jsonl',
    'CMeIE-V2_test.jsonl',
]
out_file_list = [
    'train.json',
    'dev.json',
    'test.json'
]

In [25]:
read_dir = './ori/CMeIE-V2/'
out_dir = './pipeline/CMeIE-V2'
out_dir = os.path.join(out_dir,out_name)
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
with open(os.path.join(read_dir,'53_schemas.json'),'r') as f:
    trip_types_list = json.load(f)

In [26]:
trip_types = sorted(set([data['predicate'] for data in trip_types_list]))

In [27]:
start = time.time()
print('out_name:{}'.format(out_name))
for data_file,out_file in zip(data_file_list,out_file_list):
    read_path = os.path.join(read_dir,data_file)
    out_path = os.path.join(out_dir,out_file)
    with jsonlines.open(read_path,'r') as f:
        datas = [data for data in f]
    with jsonlines.open(out_path,'w') as fw:
        for data in datas:
            inp = inp_prompt.format(relation_list='，'.join(trip_types),entity_list = '，'.join(ent_types),text=data['text'])
            spo_list = [(item['subject'],item['subject_type'],item['predicate'],item['object']['@value'],item['object_type']['@value']) for item in data['spo_list']]
            processed_spo_list = []
            for spo_item in spo_list:
                if spo_item not in processed_spo_list:
                    processed_spo_list.append(spo_item)
            oup = '\n'.join(['（{}||{}||{}||{}||{}）'.format(item[0],item[1],item[2],item[3],item[4]) for item in processed_spo_list])
            oup = '```\n' + oup.strip() + '\n```'
            out_data = {
                'instruction':inp,
                'input':'',
                'output':oup,
                'text':data['text'],
                'spo_list':data['spo_list']
            }
            fw.write(out_data)
end = time.time()
print('cost:{}秒'.format(round(end-start, 2)))

out_name:c_to_hrt_withtype
cost:1.63秒


In [28]:
all_end = time.time()
all_end-all_start

27.102506160736084